In [ ]:
import os
import json

# Function to read the JSON file and return its content
def read_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Function to create a Postman request item from JSON files
def create_request_item(api_name, operation_name, method_name, request_id):
    request = read_json(f'{api_name}_{operation_name}_{method_name}_{request_id}_Request.json')
    request_header = read_json(f'{api_name}_{operation_name}_{method_name}_{request_id}_RequestHeader.json')
    response_header = read_json(f'{api_name}_{operation_name}_{method_name}_{request_id}_ResponseHeader.json')
    response = read_json(f'{api_name}_{operation_name}_{method_name}_{request_id}_Response.json')

    resource_url = request_header['transactionResource']
    headers = request_header['headers']
    headers.append({'key': '.ASPXAUTH', 'value': '{{ASPXAUTH}}'})

    return {
        'name': f'{api_name}_{operation_name}_{method_name}',
        'request': {
            'method': method_name,
            'header': headers,
            'url': '{{baseURL}}' + resource_url,
            'body': request
        },
        'response': [{
            'code': 200,
            'header': response_header,
            'body': json.dumps(response)
        }]
    }

# Main function to generate the Postman collection
def generate_postman_collection():
    collection = {
        'info': {
            'name': 'Generated Postman Collection',
            'schema': 'https://schema.getpostman.com/json/collection/v2.1.0/collection.json'
        },
        'variable': [
            {
                'key': 'baseURL',
                'value': ''
            },
            {
                'key': 'ASPXAUTH',
                'value': ''
            }
        ],
        'item': []
    }

    api_names = set()

    for file in os.listdir():
        if '_Request.json' in file:
            api_name, operation_name, method_name, request_id, _ = file.split('_')
            api_names.add(api_name)
            request_item = create_request_item(api_name, operation_name, method_name, request_id)
            collection['item'].append(request_item)

    for api_name in api_names:
        os.makedirs(api_name, exist_ok=True)

    with open('Generated_Postman_Collection.json', 'w') as file:
        json.dump(collection, file, indent=2)

if __name__ == '__main__':
    generate_postman_collection()
